In [13]:
import pandas as pd
import numpy as np
import re
import ast

In [116]:
def truncate_string(match):
    start_index = 0
    end_index = len(match) - 1

    # Find the index of the first letter from the beginning
    while start_index <= end_index and not match[start_index].isalpha():
        start_index += 1

    # Find the index of the first letter from the end
    while end_index >= start_index and not match[end_index].isalpha():
        end_index -= 1

    # Check if there are any letters in the match
    if start_index > end_index:
        return None

    return match[start_index:end_index+1] + "."

def match_correct_regex(regexes, string_to_process):
    reviews = []
    for regex in regexes:
        result = regex.findall(string_to_process)
        if len(result) != 0:
            for match_tuple in result:
                if type(match_tuple) is tuple:
                    for match in match_tuple:
                        final_out = truncate_string(match)
                        if final_out is not None:
                            reviews.append(final_out)
                        if len(reviews) == 3:
                            return reviews
                else:
                    final_out = truncate_string(match_tuple)
                    if final_out is not None:
                        reviews.append(final_out)
                    if len(reviews) == 3:
                        return reviews
    return None

In [122]:
def split_dataframe_reviews(row, regexes):
    if type(row.chatgpt_out) is str:
        reviews = match_correct_regex(regexes, row.chatgpt_out)
        if reviews is not None:
            row.review1 = reviews[0]
            row.review2 = reviews[1]
            row.review3 = reviews[2]
    return row

In [123]:
# Testing strings
my_string = "['The movie had a lot of potential, but the CGI effects were extremely disappointing. It felt like I was watching a low-budget film with amateur graphics. The characters looked stiff and artificial, and the special effects were clearly computer-generated. It was distracting and took away from the overall experience of the film. The story was intriguing, but the poor CGI effects ruined it for me. I wouldn\'t recommend this movie to anyone looking for a visually stunning film.', 'The CGI effects in this movie were a complete letdown. The filmmakers clearly didn\'t invest enough time or money into creating believable and realistic visuals. The characters looked like they were made out of plastic and moved in an awkward and unnatural way. The action scenes were particularly disappointing, as the special effects looked fake and poorly executed. It was hard to stay engaged in the story when the visuals were so distracting. I would skip this movie if you\'re looking for a film with impressive CGI effects.', 'I was really disappointed by the CGI effects in this movie. It\'s a shame, because the story was interesting and had a lot of potential. However, the poorly executed special effects really took away from the overall experience. The characters looked stiff and robotic, and the action scenes were unconvincing. It\'s hard to get invested in a film when the visuals are so distracting. I think this movie could have been much better if the filmmakers had invested more time and resources into the CGI effects. As it stands, I wouldn\'t recommend it.']"
my_other_string = "1. Wow, what a thrilling film! I haven't been this excited to take a nap since I was in preschool. The plot was so boring and predictable that I was literally counting down the minutes until it was over. Who needs caffeine when you can just watch this film to wake yourself up in the morning? 2. Forget counting sheep, watch this film to fall asleep. It's the perfect cure for insomnia. The plot is so bland and uninteresting that I'm pretty sure my pillow had more personality than the main character. If you're looking for excitement, look elsewhere. 3. If you're struggling to fall asleep at night, look no further than this film. It's guaranteed to bore you to sleep in no time. The plot was so generic and unremarkable that I could barely keep my eyes open. It's the perfect film for those who enjoy the sensation of watching paint dry."
another_sdtring = "['This film is the epitome of generic. It follows a predictable plotline and offers nothing new or innovative to the genre. The characters are forgettable and lack any depth or complexity. The acting is wooden and unconvincing, making it difficult to become invested in the story. Overall, a forgettable and unremarkable film.'] [ 'This film is a prime example of lazy filmmaking. The plot is uninspired and lacks any originality. It feels like a cookie-cutter production, following all the generic tropes and cliches of its genre. The characters are one-dimensional and uninteresting, with no real development throughout the film. The performances are lackluster and fail to elevate the weak material. A forgettable and disappointing film.'] ['Watching this film was a tedious experience. It fails to bring anything new or exciting to the table, instead relying on tired formulas and generic plot devices. The characters are forgettable and lack any real personality, making it difficult to care about their journey. The dialogue is uninspired and lacks any real wit or charm. The film feels like a cheap knockoff of better productions, and ultimately fails to leave a lasting impression.']"

In [124]:
regex_pattern_1 = '(\[.*?\'.*?\'\s*?,)|(\s*?\'.*?\'\s*?,)|(\s*?\'.*?\' *?\])'
regex_pattern_2 = '(\[.*?\".*?\"\s*?,)|(\s*?\".*?\"\s*?,)|(\s*?\".*?\" *?\])'
regex_pattern_3 = '([123].[^123]*)'
regex1 = re.compile(regex_pattern_1)
regex2 = re.compile(regex_pattern_2)
regex3 = re.compile(regex_pattern_3)
regexes = [regex1, regex2, regex3]
pattern = '^\[[^\[\]]*\]$'
df = pd.read_csv("./data/reviews_data.tsv", sep="\t")

In [125]:
df_split = df.apply(split_dataframe_reviews, regexes=regexes, axis=1)

In [126]:
df_split

,id,prompt,chatgpt_out,review1,review2,review3,Unnamed: 6
0,0,Create three film reviews about the same film....,"['The movie had a lot of potential, but the CG...","The movie had a lot of potential, but the CGI ...",The CGI effects in this movie were a complete ...,I was really disappointed by the CGI effects i...,0.0
1,1,Create three film reviews about the same film....,['The film was so badly shot that I wondered i...,The film was so badly shot that I wondered if ...,The cinematographer of this film must have bee...,The cinematography in this film was so bad tha...,NaN
2,2,Create three film reviews about the same film....,"['The film had potential, but unfortunately th...","The film had potential, but unfortunately the ...",I was really disappointed with this film. The ...,The film had a decent cast and a promising sto...,NaN
3,3,Create three film reviews about the same film....,"['The film had a great potential, but unfortun...","The film had a great potential, but unfortunat...","The pacing of the film was painfully slow, and...","I was disappointed with the film, as the trail...",NaN
4,4,Create three film reviews about the same film....,"[ ""Despite the hype, this film falls short of ...","Despite the hype, this film falls short of exp...",This film is a perfect example of style over s...,"While the film has a lot of potential, it ulti...",NaN
...,...,...,...,...,...,...,...
12495,12495,Create three film reviews about the same film....,NaN,NaN,NaN,NaN,NaN
12496,12496,Create three film reviews about the same film....,NaN,NaN,NaN,NaN,NaN
12497,12497,Create three film reviews about the same film....,NaN,NaN,NaN,NaN,NaN
12498,12498,Create three film reviews about the same film....,NaN,NaN,NaN,NaN,NaN


In [128]:
df_split.to_csv("./data/reviews_data_split.csv", sep=";")